<a href="https://colab.research.google.com/github/Aminfaraji/ABINetpaddleocr/blob/main/salhe_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import numpy as np
!pip install torchmetrics
!sudo apt-get install tesseract-ocr-fas
!pip install pytesseract
from torchmetrics.text import CharErrorRate ,WordErrorRate
import pytesseract
from glob import glob
import shutil,os,cv2
from difflib import SequenceMatcher
from PIL import Image
from tqdm import tqdm
!pip install easyocr
import easyocr

In [29]:
from IPython.display import Image
import requests

def show_image(image_name):
  return Image(filename=image_name)
reader_en_fr = easyocr.Reader(['en', 'fa'])

def read_text(image_name, model_name, in_line=False):

  # Read the data
  text = model_name.readtext(image_name, detail = 0, paragraph=in_line)

  return '\n'.join(text)

In [30]:
# show_image("/content/drive/MyDrive/images/001A10.jpg")
fr_text = read_text("/content/drive/MyDrive/images/001A10.jpg", reader_en_fr)
print(fr_text)

KeyboardInterrupt: 

In [20]:
import xml.etree.ElementTree as ET
from PIL import Image
import pytesseract
import pandas as pd
import os

def extract_texts_with_bndbox(element):
    results = []
    for obj in element.findall('.//object'):
        content_element = obj.find('content')
        bndbox_element = obj.find('bndbox')
        if content_element is not None and bndbox_element is not None:
            text = content_element.text.strip()
            bndbox = {
                'xmin': int(bndbox_element.find('xmin').text),
                'ymin': int(bndbox_element.find('ymin').text),
                'xmax': int(bndbox_element.find('xmax').text),
                'ymax': int(bndbox_element.find('ymax').text),
            }
            results.append((text, bndbox))
    return results

# برش تصاویر
def crop_image(image_path, bndbox):
    image = Image.open(image_path)
    cropped_image = image.crop((bndbox['xmin'], bndbox['ymin'], bndbox['xmax'], bndbox['ymax']))
    return cropped_image

# انجام OCR با Tesseract
def ocr_image_tesseract(image):
    text = pytesseract.image_to_string(image, lang='eng+fa')
    return text

# انجام OCR با EasyOCR
def ocr_image_easyocr(reader, image):
  result = reader.readtext(image , paragraph=True)
  text = ' '.join([res[1] for res in result])
  return text

# محاسبه نرخ خطا
def calculate_error_rates(ground_truth, ocr_text):
    char_error_rate = cer(ocr_text , ground_truth)
    word_error_rate = wer(ocr_text , ground_truth)
    return char_error_rate, word_error_rate

# پردازش هر فایل XML
def process_xml_file(xml_path, image_path, reader):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # استخراج متن‌ها و مختصات‌ها
    texts_with_bndbox = extract_texts_with_bndbox(root)

    results = []
    for text, bndbox in texts_with_bndbox:
        cropped_image = crop_image(image_path, bndbox)

        ocr_text_tesseract = ocr_image_tesseract(cropped_image)
        ocr_text_easyocr = ocr_image_easyocr(reader, cropped_image)

        char_error_rate_tesseract, word_error_rate_tesseract = calculate_error_rates(text, ocr_text_tesseract)
        char_error_rate_easyocr, word_error_rate_easyocr = calculate_error_rates(text, ocr_text_easyocr)

        # ذخیره اطلاعات در لیست
        results.append({
            'Original Text': text,
            'OCR Text Tesseract': ocr_text_tesseract,
            'OCR Text EasyOCR': ocr_text_easyocr,
            'Bounding Box': f"{bndbox['xmin']},{bndbox['ymin']},{bndbox['xmax']},{bndbox['ymax']}",
            'Character Error Rate Tesseract': char_error_rate_tesseract.item(),
            'Word Error Rate Tesseract': word_error_rate_tesseract.item(),
            'Character Error Rate EasyOCR': char_error_rate_easyocr.item(),
            'Word Error Rate EasyOCR': word_error_rate_easyocr.item(),
            'Image Path': os.path.basename(image_path),
            'XML File': os.path.basename(xml_path)
        })
    return results

xml_files =glob('/content/drive/MyDrive/ArnaDataset/*.xml')
image_files = '/content/drive/MyDrive/images/'
all_data = []
reader = easyocr.Reader(['en' , 'fa'])


cer = CharErrorRate()
wer = WordErrorRate()
for xml in tqdm(xml_files):
  all_data.extend(process_xml_file(xml,image_files+os.path.split(xml)[1][:-4]+'.jpg' , reader))

df = pd.DataFrame(all_data)
df.to_excel('output.xlsx', index=False)

print("Data has been saved to output.xlsx")


  0%|          | 0/1001 [00:00<?, ?it/s]


ValueError: Invalid input type. Supporting format = string(file path or url), bytes, numpy array

In [1]:
# df = pd.DataFrame(all_data)
# df

'/content/drive/MyDrive/images/011C9.jpg'

In [72]:
xml_files

['/content/drive/MyDrive/ArnaDataset/011C9.xml',
 '/content/drive/MyDrive/ArnaDataset/011C11.xml',
 '/content/drive/MyDrive/ArnaDataset/011C2.xml',
 '/content/drive/MyDrive/ArnaDataset/011C12.xml',
 '/content/drive/MyDrive/ArnaDataset/011C3.xml',
 '/content/drive/MyDrive/ArnaDataset/011C18.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB3.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB6.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB8.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB9.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB11.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB12.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB13.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB14.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB15.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB17.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB18.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB19.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB20.xml',
 '/content/drive/MyDrive/ArnaDataset/111TB21